In [ ]:
# Reload all src modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import cProfile
import pandas as pd
import geopandas as geopd
import numpy as np
import multiprocessing as mp
import re
import gzip
try:
    import cld3
except ModuleNotFoundError:
    pass
import pycld2
from pyproj import Transformer
from shapely.geometry import Polygon
from shapely.geometry import Point
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import descartes
import folium
import src.utils.geometry as geo
import src.data.shp_extract as shp_extract
import src.data.tweets_cells_counts as tweets_counts
import src.data.text_process as text_process
import src.data.access as data_access
import src.visualization.grid_viz as grid_viz
import src.data.user_filters as ufilters
from dotenv import load_dotenv
load_dotenv()

pd.reset_option("display.max_rows")

In [ ]:
data_dir_path = os.environ['DATA_DIR']
tweets_files_format = 'tweets_2015_2018_{}.json.gz'
places_files_format = 'places_2015_2018_{}.json.gz'
ssh_domain = os.environ['IFISC_DOMAIN']
ssh_username = os.environ['IFISC_USERNAME']
country_codes = ('BO', 'CA', 'CH', 'EE', 'ES', 'FR', 'HK','ID', 'LT', 'LV',
                'MY', 'PE', 'RO', 'SG', 'TN', 'UA')
latlon_proj = 'epsg:4326'
xy_proj = 'epsg:3857'
external_data_dir = '../data/external/'
fig_dir = '../reports/figures'
cc = 'CH'

In [ ]:
tweets_file_path = os.path.join(data_dir_path, tweets_files_format.format(cc))
raw_tweets_df_generator = data_access.yield_json(tweets_file_path, 
    ssh_domain=ssh_domain, ssh_username=ssh_username, chunk_size=100000, compression='gzip')
for i,raw_tweets_df in enumerate(raw_tweets_df_generator):
    break

raw_tweets_df_generator.close()
raw_tweets_df.head()

In [ ]:
places_file_path = os.path.join(data_dir_path, places_files_format.format(cc))
shapefile_name = 'CNTR_RG_01M_2016_4326.shp'
shapefile_path = os.path.join(external_data_dir, shapefile_name, shapefile_name)
shape_df = geopd.read_file(shapefile_path)
shape_df = shape_df.loc[shape_df['FID'] == cc]
raw_places_df = data_access.return_json(places_file_path, 
    ssh_domain=ssh_domain, ssh_username=ssh_username, compression='gzip')
raw_places_df.head()

In what seems to be most countries, places include very few poi, for instance in CH, there are only two of them, in PE none

Get most frequent, small enough place: if most frequent -> select it, if within more frequent bigger place -> select it, 

If not small enough place, discard the user

In [ ]:
print(raw_tweets_df.info())
nr_users = len(raw_tweets_df['uid'].unique())
print('There are {} distinct users in the dataset'.format(nr_users))

The "I'm at \<place\>" from Foursquare are also there, and they all have 'source' = <a href="http://foursquare.com" rel="nofollow">Foursquare</a>. Tweetbot is an app for regular users, it's not related to bot users.

In [ ]:
tweets_df = raw_tweets_df[['text', 'id', 'lang', 'place_id', 'coordinates', 'uid', 'created_at']]
tweets_df = tweets_df.rename(columns={'lang': 'twitter_lang'})
null_reply_id = 'e39d05b72f25767869d44391919434896bb055772d7969f74472032b03bc18418911f3b0e6dd47ff8f3b2323728225286c3cb36914d28dc7db40bdd786159c0a'
raw_tweets_df.loc[raw_tweets_df['in_reply_to_status_id'] == null_reply_id, 
    ['in_reply_to_status_id', 'in_reply_to_screen_name', 'in_reply_to_user_id']] = None
tweets_df['source'] = raw_tweets_df['source'].str.extract(r'>(.+)</a>', expand=False)
tweets_df['source'].value_counts().head(20)

### Get residents

In [ ]:
tweets_file_path = os.path.join(data_dir_path, tweets_files_format.format(cc))

def pre_process(tweets_file_path, chunk_start, chunk_size):
    raw_tweets_df = data_access.read_json_wrapper(tweets_file_path, chunk_start, chunk_size, 
                                                  ssh_domain=ssh_domain, ssh_username=ssh_username)
    tweeted_months_users = ufilters.get_months_activity(raw_tweets_df)
    return raw_tweets_df, tweeted_months_users

def profile_pre_process(tweets_file_path, chunk_start, chunk_size):
    cProfile.runctx('pre_process(tweets_file_path, chunk_start, chunk_size)', globals(), locals())

agg_tweeted_months_users = pd.DataFrame([], columns=['uid', 'month'])
with mp.Pool(4) as pool:
    jobs = []
    for chunk_start, chunk_size in data_access.chunkify(tweets_file_path, size=1e9, 
                                                        ssh_domain=ssh_domain, ssh_username=ssh_username):
        jobs.append(pool.apply_async(pre_process, (tweets_file_path, chunk_start, chunk_size)))

    for job in jobs:
        raw_tweets_df, tweeted_months_users = job.get()
        agg_tweeted_months_users = pd.concat([agg_tweeted_months_users, tweeted_months_users])
total_nr_users = len(agg_tweeted_months_users['uid'].unique())
print('In total, there are {} distinct users in the whole dataset'.format(total_nr_users))

In [ ]:
local_uid_series = ufilters.consec_months(agg_tweeted_months_users)
crs = {'init': latlon_proj}
places_df = raw_places_df[['id', 'bounding_box', 'name', 'place_type']]
geometry = places_df['bounding_box'].apply(lambda x: Polygon(x['coordinates'][0]))
places_geodf = geopd.GeoDataFrame(places_df, crs=crs, geometry=geometry)
places_geodf = places_geodf.set_index('id')
places_geodf = places_geodf.drop(columns=['bounding_box'])
places_geodf['area'] = places_geodf.geometry.to_crs(xy_proj).area
print('done')

In [ ]:
def process(raw_tweets_df, local_uid_series, places_geodf, text_col='text', min_nr_words=4, cld='pycld2'):
    tweets_df = raw_tweets_df[['text', 'id', 'lang', 'place_id', 'coordinates', 'uid', 'created_at']]
    tweets_df = tweets_df.loc[tweets_df['uid'].isin(local_uid_series.index.values)]
    tweets_df = tweets_df.rename(columns={'lang': 'twitter_lang'})
    print('starting lang detect')
    tweets_lang_df = text_process.lang_detect(tweets_df, text_col='text', min_nr_words=4, cld='pycld2')
    tweets_lang_df = tweets_lang_df.loc[tweets_lang_df['cld_lang'] != 'un']
    tweets_to_loc_df = tweets_lang_df.loc[:]
    tweets_final_df = tweets_to_loc_df.join(places_geodf, on='place_id', how='left')
    print('processing done')
    return tweets_final_df

def profile_process(raw_tweets_df, local_uid_series, places_geodf):
    cProfile.runctx('process(raw_tweets_df, local_uid_series, places_geodf)', globals(), locals())

list_tweets_df = []
with mp.Pool(8) as pool:
    results = []
    for job in jobs:
        raw_tweets_df = job.get()[0]
        results.append(pool.apply_async(process, (raw_tweets_df, local_uid_series, places_geodf)))

    for res in results:
        res.get()

In [ ]:
cell_size = 5000
cells_df, cells_in_area_df = geo.create_grid(shape_df, cell_size, latlon_proj, xy_proj, intersect=True)
grid_test_df = cells_in_area_df.copy()
grid_test_df['metric'] = 1
save_path = os.path.join(fig_dir, 'grid_cc={}_cell_size={}m.pdf'.format(cc, cell_size))
plot_kwargs = dict(alpha=0.7, edgecolor='w', linewidths=0.001)
grid_viz.plot_grid(grid_test_df, shape_df, metric_col='metric', save_path=save_path, **plot_kwargs)

Few users speaking Spanish but lots of tweets, must be a bot there

pb: overlapping places' bbox

In [ ]:
plot_lang = 'es'
max_place_area = 1e9
agg_count_df = pd.Series(name='count', dtype='int64')
for res in results:
    df = res.get()
    df = df.loc[(df['area'] < max_place_area) & (df['cld_lang'] == plot_lang)]
    agg_count_df = tweets_counts.increment_counts(agg_count_df, df,
                                                 'place_id')
places_counts_df = places_geodf.join(agg_count_df, how='inner')
print('There are {:n} tweets in {}.'.format(agg_count_df.sum(), plot_lang))

In [ ]:
cells_in_area_df['cell_id'] = cells_in_area_df.index
cells_intersect_places = geopd.overlay(places_counts_df, cells_in_area_df, how='intersection')
cells_intersect_places['area_intersect'] = cells_intersect_places.geometry.to_crs(xy_proj).area
cells_intersect_places['cell_count'] = cells_intersect_places['count'] * (cells_intersect_places['area_intersect']
                                                                          / cells_intersect_places['area'])
cells_counts = cells_intersect_places.groupby('cell_id')['cell_count'].sum()  
cell_plot_df = cells_in_area_df.join(cells_counts, how='inner')

In [ ]:
save_path = os.path.join(fig_dir, 
    'tweet_counts_cc={}_lang={}_cell_size={}m.pdf'.format(cc, plot_lang, cell_size))
country_name = shape_df['NAME_ENGL'].values[0]
langs_dict = dict(pycld2.LANGUAGES)
for key in langs_dict:
    if langs_dict[key] == plot_lang:
        formatted_lang = key.lower().capitalize()  
plot_title = 'Distribution of {} speakers in {}'.format(formatted_lang, country_name)
cbar_label = 'Number of tweets in the cell'
plot_kwargs = dict(edgecolor='w', linewidths=0.001, cmap='Purples')
ax = grid_viz.plot_grid(cell_plot_df, shape_df, metric_col='cell_count', save_path=save_path, 
                        title=plot_title, cbar_label=cbar_label, **plot_kwargs)

In [ ]:
start_point = shape_df['geometry'].values[0].centroid
m = folium.Map(location=[start_point.y, start_point.x], zoom_start=7, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=cell_plot_df,
    name='choropleth',
    data=cell_plot_df['cell_count'],
    key_on='feature.properties.cell_id',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=1,
    line_color='w',
    legend_name=cbar_label
).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
plot_df = places_counts_df.copy()
plot_df['ratio'] = plot_df['count'] / plot_df['count'].max()
save_path = os.path.join(fig_dir, 
    'tweet_counts_cc={}_lang={}_cell_size={}m.pdf'.format(cc, plot_lang, cell_size))
plot_kwargs = dict(alpha=0.7, edgecolor='w', legend=True, linewidths=0.001)
ax = grid_viz.plot_grid(plot_df, shape_df, metric_col='ratio', save_path=save_path, show=False, **plot_kwargs)

### FIrst tests on single df

In [ ]:
ref_year = 2015
nr_consec_months = 3
tweets_file_path = os.path.join(data_dir_path, tweets_files_format.format(cc))
raw_tweets_df_generator = data_access.yield_json(tweets_file_path, 
    ssh_domain=ssh_domain, ssh_username=ssh_username, chunk_size=1000000, compression='gzip')
agg_tweeted_months_users = pd.DataFrame([], columns=['uid', 'month'])
tweets_df_list = []
for raw_tweets_df in raw_tweets_df_generator:
    tweets_df_list.append(raw_tweets_df)
    tweeted_months_users = ufilters.get_months_activity(raw_tweets_df)
    agg_tweeted_months_users = pd.concat([agg_tweeted_months_users, tweeted_months_users])
raw_tweets_df_generator.close()
local_uid_series = ufilters.consec_months(agg_tweeted_months_users)

In [ ]:
ref_year = 2015
nr_consec_months = 3
tweeted_months_users = ufilters.get_months_activity(tweets_df)
local_uid_series = ufilters.consec_months(tweeted_months_users)

In [ ]:
pd.DataFrame(local_uid_series)

In [ ]:
raw_tweets_df['lang'].value_counts().head(10)

In [ ]:
raw_tweets_df.join(local_uid_series, on='uid', how='inner')['lang'].value_counts().head(10)

In [ ]:
tweets_file_path = os.path.join(data_dir_path, tweets_files_format.format(cc))
raw_tweets_df_generator = data_access.yield_json(tweets_file_path, 
    ssh_domain=ssh_domain, ssh_username=ssh_username, chunk_size=1000000, compression='gzip')
for raw_tweets_df in raw_tweets_df_generator:
    filtered_tweets_df = pd.DataFrame(local_uid_series

### Language detection

- Languages possibly detected by CLD:

In [ ]:
lang_with_code = dict(pycld2.LANGUAGES)
detected_lang_with_code = [(lang, lang_with_code[lang]) for lang in pycld2.DETECTED_LANGUAGES]
print(detected_lang_with_code)

- Languages possibly detected by Twitter (see 'lang' in https://support.gnip.com/apis/powertrack2.0/rules.html#Operators):

Amharic - am
Arabic - ar
Armenian - hy
Bengali - bn
Bulgarian - bg
Burmese - my
Chinese - zh
Czech - cs
Danish - da
Dutch - nl
English - en
Estonian - et
Finnish - fi
French - fr
Georgian - ka
German - de
Greek - el
Gujarati - gu
Haitian - ht
Hebrew - iw
Hindi - hi
Hungarian - hu
Icelandic - is
Indonesian - in
Italian - it
Japanese - ja
Kannada - kn
Khmer - km
Korean - ko
Lao - lo
Latvian - lv
Lithuanian - lt
Malayalam - ml
Maldivian - dv
Marathi - mr
Nepali - ne
Norwegian - no
Oriya - or
Panjabi - pa
Pashto - ps
Persian - fa
Polish - pl
Portuguese - pt
Romanian - ro
Russian - ru
Serbian - sr
Sindhi - sd
Sinhala - si
Slovak - sk
Slovenian - sl
Sorani Kurdish - ckb
Spanish - es
Swedish - sv
Tagalog - tl
Tamil - ta
Telugu - te
Thai - th
Tibetan - bo
Turkish - tr
Ukrainian - uk
Urdu - ur
Uyghur - ug
Vietnamese - vi
Welsh - cy

In [ ]:
tweets_lang_df = text_process.lang_detect(tweets_df, text_col='text', min_nr_words=4, cld='pycld2')
tweets_lang_df.head()

In [ ]:
cld_langs = tweets_lang_df['cld_lang'].unique()
cld_langs.sort()
print('Languages detected by cld: {}'.format(cld_langs))
twitter_langs = tweets_lang_df['twitter_lang'].unique()
twitter_langs.sort()
print('Languages detected by twitter: {}'.format(twitter_langs))

In [ ]:
tweets_lang_df['twitter_lang'].value_counts().head(10)

In [ ]:
tweets_lang_df['cld_lang'].value_counts().head(10)

French case, corsican is unreliably detected by CLD for French tweets, however seems pretty accurate when twitter_lang='it'

Let's look at multilingual users

In [ ]:
groupby_user_lang = tweets_lang_df.loc[tweets_lang_df['twitter_lang'] != 'und'].groupby(['uid', 'twitter_lang'])
count_tweets_by_user_lang = groupby_user_lang.size()
count_langs_by_user_df = count_tweets_by_user_lang.groupby('uid').transform('size')
multiling_users_df = count_tweets_by_user_lang.loc[count_langs_by_user_df > 1]
pd.DataFrame(multiling_users_df)

In [ ]:
pd.set_option("display.max_rows", 100)
multiling_users_list = [x[0] for x in multiling_users_df.index.values]
tweets_lang_df[tweets_lang_df['uid'].isin(multiling_users_list)].sort_values(by=['uid', 'cld_lang'])[
    ['uid', 'filtered_text', 'cld_lang', 'twitter_lang', 'created_at']]

### Places into geodf and join on tweets

Calculate the area to discard bbox which are too large? Problem: need to project first, which is expensive

In [ ]:
tweets_to_loc_df = tweets_lang_df.loc[tweets_lang_df['coordinates'].isnull()]
crs = {'init': latlon_proj}
places_df = raw_places_df[['id', 'bounding_box', 'name', 'place_type']]
geometry = places_df['bounding_box'].apply(lambda x: Polygon(x['coordinates'][0]))
places_geodf = geopd.GeoDataFrame(places_df, crs=crs, geometry=geometry)
places_geodf = places_geodf.set_index('id')
places_geodf = places_geodf.drop(columns=['bounding_box'])
places_geodf['area'] = places_geodf.geometry.to_crs(xy_proj).area
tweets_final_df = tweets_to_loc_df.join(places_geodf, on='place_id', how='left')
tweets_final_df.head(10)

In [ ]:
tweets_final_df.loc[(tweets_final_df['cld_lang'] =='co') & (tweets_final_df['twitter_lang'] =='it')]

CLD sensitive to letter repetitions made to insist: can put threshold if more than 3 consecutive same letter, bring it down to 2, it seems to improve prediction on example

Usually twitter's prediction seems better...

In [ ]:
tweets_final_df[tweets_final_df['cld_lang'] != tweets_final_df['twitter_lang']].drop(columns=['id'])

### groupbys and stuff

In [ ]:
def get_mean_time(df, dt_col):
    t_series_in_sec_of_day = df['hour']*3600 + df['minute']*60 + df['second']
    return pd.to_timedelta(int(t_series_in_sec_of_day.mean()), unit='s')

In [ ]:
tweets_df = raw_tweets_df.copy()
# Speeds up the process to extract the hour, min and sec first
tweets_df['hour'] = tweets_df['created_at'].dt.hour
tweets_df['minute'] = tweets_df['created_at'].dt.minute
tweets_df['second'] = tweets_df['created_at'].dt.second
groupby_user_place = tweets_df.groupby(['uid', 'place_id'])
count_tweets_by_user_place = groupby_user_place.size()
count_tweets_by_user_place.rename('count', inplace=True)
mean_time_by_user_place = groupby_user_place.apply(lambda df: get_mean_time(df, 'created_at'))
mean_time_by_user_place.rename('avg time', inplace=True)
# transform to keep same size, so as to be able to have a matching boolean Series of same size as 
# original df to select users with more than one place for example:
count_places_by_user_df = count_tweets_by_user_place.groupby('uid').transform('size')
agg_data_df = pd.concat([count_tweets_by_user_place, mean_time_by_user_place], axis=1)
count_tweets_by_user_place_geodf = agg_data_df.join(places_geodf, on='place_id')
count_tweets_by_user_place_geodf.head()

In [ ]:
cProfile.run("groupby_user_place.apply(lambda df: get_mean_time(df, 'created_at'))")

In [ ]:
count_tweets_by_user_place_geodf.loc[count_places_by_user_df > 1]

Add new chunk to cumulative data:

In [ ]:
count_tweets_by_user_place_geodf = count_tweets_by_user_place_geodf.join(
    count_tweets_by_user_place_geodf['count'], 
    on=['uid', 'place_id'], how='outer', rsuffix='_new')
count_tweets_by_user_place_geodf['count'] += count_tweets_by_user_place_geodf['count_new']
count_tweets_by_user_place_geodf.drop(columns=['count_new'], inplace=True)
count_tweets_by_user_place_geodf